In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import mutual_info_score
import tensorflow as tf

Preprocess the data by encoding categorical variables and removing null values.\

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv')
df['score_text'] = df['score_text'].replace({'High': 2, 'Medium': 1, 'Low': 0})
df['score_text'] = df['score_text'].fillna(0)
df['v_score_text'] = df['v_score_text'].replace({'High': 2, 'Medium': 1, 'Low': 0})
df['v_score_text'] = df['v_score_text'].fillna(0)
df = df[df['race'].isin(['Caucasian', 'African-American'])]
df['race'] = df['race'].replace({'Caucasian': 1, 'African-American': 0})
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

Make a Logistic Regression model that predicts two_year_recidivism without any prejudice remover. Notice that without any prejudice removal, the model predicts recidivism more accurately for Caucasians compared to African Americans.

In [73]:
X = df[['id', 'age', 'juv_fel_count', 'juv_misd_count', 'is_recid', 'decile_score','juv_other_count', 'priors_count', 'v_score_text', 'is_violent_recid', 'race']].copy()
X = X.fillna(0)
Y = df['two_year_recid'].copy()
unfair_model = LogisticRegression(max_iter=1000)
unfair_model.fit(X, Y)

priv = X[X['race'] == 1]
unpriv = X[X['race'] == 0]

priv_pred = unfair_model.predict(priv)
unpriv_pred = unfair_model.predict(unpriv)

accuracy_priv = accuracy_score(Y[X['race'] == 1], priv_pred)
accuracy_unpriv = accuracy_score(Y[X['race'] == 0], unpriv_pred)

print("Accuracy for privileged group (race == 1):", accuracy_priv)
print("Accuracy for unprivileged group (race == 0):", accuracy_unpriv)


Accuracy for privileged group (race == 1): 0.9759576202118989
Accuracy for unprivileged group (race == 0): 0.963474025974026


Store the class Y, the non-sensitive features X, and the sensitive feature S separately.

In [56]:
non_sensitive_features = ['id', 'age', 'juv_fel_count', 'juv_other_count', 'priors_count', 'v_score_text', 'is_violent_recid']
X = df[non_sensitive_features].copy()
Y = df['two_year_recid'].copy()
S = df['race'].copy()
print(X)
print(Y)
print(S)

2454


In [113]:
def PRLOSS(unpriv, priv, eta):
    unpriv_float = tf.cast(unpriv, dtype=tf.float32)
    priv_float = tf.cast(priv, dtype=tf.float32)

    n_unpriv = tf.cast(tf.shape(unpriv_float)[0], dtype=tf.float32)
    n_priv = tf.cast(tf.shape(priv_float)[0], dtype=tf.float32)

    Dxisi = tf.cast(tf.stack([n_priv, n_unpriv], axis=0), tf.float32)

    y_pred_priv = tf.reduce_sum(n_priv)
    y_pred_unpriv = tf.reduce_sum(n_unpriv)

    P_ys_stacked = tf.stack([y_pred_priv, y_pred_unpriv], axis=0)
    P_ys = P_ys_stacked / Dxisi

    P = tf.concat([unpriv_float, priv_float], axis=0)

    P_sum = tf.reduce_sum(P)
    total_samples = tf.cast(tf.size(unpriv_float) + tf.size(priv_float), dtype=tf.float32)
    P_y = P_sum / total_samples

    log_P_ys_1 = tf.math.log(P_ys[1])
    log_P_y = tf.math.log(P_y)
    P_s1y1 = log_P_ys_1 - log_P_y

    log_1_minus_P_ys_1 = tf.math.log(1 - P_ys[1])
    log_1_minus_P_y = tf.math.log(1 - P_y)
    P_s1y0 = log_1_minus_P_ys_1 - log_1_minus_P_y

    log_P_ys_0 = tf.math.log(P_ys[0])
    log_P_y = tf.math.log(P_y)
    P_s0y1 = log_P_ys_0 - log_P_y

    log_1_minus_P_ys_0 = tf.math.log(1 - P_ys[0])
    log_1_minus_P_y = tf.math.log(1 - P_y)
    P_s0y0 = log_1_minus_P_ys_0 - log_1_minus_P_y

    P_s1y1 = tf.reshape(P_s1y1, [-1])
    P_s1y0 = tf.reshape(P_s1y0, [-1])
    P_s0y1 = tf.reshape(P_s0y1, [-1])
    P_s0y0 = tf.reshape(P_s0y0, [-1])

    PI_s1y1 = unpriv_float * P_s1y1
    PI_s1y0 = (1 - unpriv_float) * P_s1y0
    PI_s0y1 = priv_float * P_s0y1
    PI_s0y0 = (1 - priv_float) * P_s0y0
    PI = tf.reduce_sum(PI_s1y1) + tf.reduce_sum(PI_s1y0) + tf.reduce_sum(PI_s0y1) + tf.reduce_sum(PI_s0y0)

    return eta * PI


Split data for training and testing.

In [105]:
X_train, X_test, S_train, S_test, Y_train, Y_test = train_test_split(X,Y,S, test_size=0.1, random_state=42)

In [115]:
eta = 0.01
def prediction_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(input_shape,))
    ])
    return model
model = prediction_model(X_train.shape[1])

model.compile(optimizer='adam', loss=lambda y_true, y_pred: pr_loss(y_true, y_pred, eta))

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))


Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "<ipython-input-115-98e347c35c01>", line 10, in None  *
        lambda y_true, y_pred: pr_loss(y_true, y_pred, eta)
    File "<ipython-input-114-408a1f72a10b>", line 15, in pr_loss  *
        N_female = tf.constant(output_f.shape[0], dtype=tf.float32)

    ValueError: None values not supported.
